<a href="https://colab.research.google.com/github/jnardosp/mc_asymptotic_analysis/blob/main/funcion_recursiva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>